In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pubchempy as pcp
import requests

In [6]:
pharmgkb_df = pd.read_csv('rawData/PharmGKB/clinicalVariants/clinicalVariants.tsv', sep='\t')
pharmgkb_df = pharmgkb_df.dropna(subset=['chemicals'])
pharmgkb_df = pharmgkb_df.dropna(subset=['gene']).reset_index(drop= True)
# print(pharmgkb_df)
pharmgkb_df.head()

,variant,gene,type,level of evidence,chemicals,phenotypes
0,"CYP2C9*1, CYP2C9*2, CYP2C9*3, CYP2C9*13",CYP2C9,Metabolism/PK,1A,meloxicam,NaN
1,"CYP2C9*1, CYP2C9*3, CYP2C9*13",CYP2C9,Metabolism/PK,1A,lornoxicam,NaN
2,"CYP2C9*1, CYP2C9*2, CYP2C9*3",CYP2C9,Metabolism/PK,1A,siponimod,NaN
3,rs17376848,DPYD,Toxicity,1A,capecitabine,Neoplasms
4,rs2297595,DPYD,Toxicity,1A,capecitabine,Neoplasms


In [7]:
chemicals_df = pd.read_csv('rawData/PharmGKB/chemicals/chemicals.tsv', sep='\t', dtype={'PubChem Compound Identifiers': str})
chemicals_df.head()


,PharmGKB Accession Id,Name,Generic Names,Trade Names,Brand Mixtures,Type,Cross-references,SMILES,InChI,Dosing Guideline,...,VIP Count,Dosing Guideline Sources,Top Clinical Annotation Level,Top FDA Label Testing Level,Top Any Drug Label Testing Level,Label Has Dosing Info,Has Rx Annotation,RxNorm Identifiers,ATC Identifiers,PubChem Compound Identifiers
0,PA166250381,10-desmethyl alpha-dihydrotetrabenazine,"""(2R,3R,11bR)-9-methoxy-3-(2-methylpropyl)-2,3...",NaN,NaN,Metabolite,PubChem Compound:59272813,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59272813
1,PA166250405,10-desmethyl beta-dihydrodeutetrabenazine,"""(2S,3R,11bR)-3-(2-methylpropyl)-9-(trideuteri...",NaN,NaN,Metabolite,PubChem Compound:117693284,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117693284
2,PA166250403,10-desmethyl beta-dihydrotetrabenazine,"""(2S,3R,11bR)-9-methoxy-3-(2-methylpropyl)-2,3...",NaN,NaN,Metabolite,PubChem Compound:59272796,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59272796
3,PA166178620,10-formyltetrahydrofolate,NaN,NaN,NaN,Biological Intermediate,PubChem Compound:10,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
4,PA166131395,10-hydroxy r-warfarin,NaN,NaN,NaN,Metabolite,HMDB:HMDB13884,NaN,NaN,No,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
variant_df = pd.read_csv('rawData/PharmGKB/variants/variants.tsv', sep='\t')
columns = ['Variant Name', 'Gene Symbols', 'Synonyms']
variant_df = pd.DataFrame(data=variant_df, columns=columns)
variant_df = variant_df.dropna(subset=['Gene Symbols'])
variant_df = variant_df[variant_df['Synonyms'].str.contains('NP')].reset_index(drop=True)

protein_dict = {
    'Gly':'G', 'Ala':'A', 'Val':'V', 'Leu':'L', 'Ile':'I',
    'Pro':'P', 'Phe':'F', 'Tyr':'Y', 'Trp':'W', 'Ser':'S', 
    'Thr':'T', 'Cys':'C', 'Met':'M', 'Asn':'N', 'Gln':'Q',
    'Asp':'D', 'Glu':'E', 'Lys':'K', 'Arg':'R', 'His':'H'
}

# split NPnumber
variant_table = pd.DataFrame(columns=['gene', 'rsid', 'variant', 'NPid'])
for i in range(variant_df.shape[0]): # 
    gene = variant_df['Gene Symbols'][i].split(',')[0]
    rsid = variant_df['Variant Name'][i]
    synonyms = variant_df['Synonyms'][i]
    # split and filtering
    syn_list = synonyms.split(',')
    np_list = [n for n in syn_list if 'NP' in n]
    missense_list = [n for n in np_list if '=' not in n]
    #print(missense_list)
    try: # for = in NPid: not missense
        NPid = missense_list[0].strip()
    except IndexError:
        continue
    string = NPid.split('p.')[-1]
    try:
        variant = protein_dict[string[0:3]] + string[3:-3] + protein_dict[string[-3:]]
    except KeyError:
        continue
    variant_table = variant_table.append([{'gene':gene, 'rsid':rsid, 'variant':variant, 'NPid':NPid}], ignore_index=True)
variant_table

,gene,rsid,variant,NPid
0,CYP1B1,rs10012,R48G,NP_000095.2:p.Arg48Gly
1,ABCG2,rs1002012563,T512N,NP_004818.2:p.Thr512Asn
2,GABRP,rs10036156,L7V,NP_055026.1:p.Leu7Val
3,SLC5A7,rs1013940,I89V,NP_068587.1:p.Ile89Val
4,UGT1A10,rs10187694,E139K,NP_061948.1:p.Glu139Lys
...,...,...,...,...
1545,CYP2A6,rs199916117,K194E,NP_000753.3:p.Lys194Glu
1546,CYP3A4,rs201821708,Y319C,NP_059488.2:p.Tyr319Cys
1547,CYP2A6,rs28399440,F118L,NP_000753.3:p.Phe118Leu
1548,CYP2C9,rs370100007,Q214H,NP_000762.2:p.Gln214His


In [9]:
# query_variant_df = pd.DataFrame()
# for i in range(len(variant_df)):
#     variant = variant_df['Variant Name'][i]
#     query_variant_df = query_variant_df.append([{'type':'dbsnp', 'variant': variant}], ignore_index = True)

# query_variant_df.to_csv('middlefile/pharmgkb_query_variant_table.tsv', sep = '\t', index=False)

gen_coord_df = pd.read_csv('middlefile/pharmgkb_gen_coords_3a9b7334.txt', sep= '\t')
gen_coord_df

,Variation ID,dbSNP,Chromosome,Position,REF Allele,ALT Allele (IUPAC),Minor Allele,Minor Allele Global Frequency,Contig,Contig Position,Band
0,rs7414551,rs7414551,1,964594,G,A,A,0.205471,GL000006.2,378606,p36.33
1,rs146898897,rs146898897,1,1582436,C,T,T,0.007388,GL000006.2,996448,p36.33
2,rs13303344,rs13303344,1,2016961,C,W,C,0.455072,GL000006.2,1430973,p36.33
3,rs4481796,rs4481796,1,2017366,T,C,T,0.216454,GL000006.2,1431378,p36.33
4,rs2376805,rs2376805,1,2024923,G,A,G,0.224441,GL000006.2,1438935,p36.33
...,...,...,...,...,...,...,...,...,...,...,...
6819,rs72554665,rs72554665,X,154532269,C,D,A,0.001589,GL000172.2,10056663,q28
6820,rs2230037,rs2230037,X,154532439,A,G,A,0.215894,GL000172.2,10056833,q28
6821,rs5030868,rs5030868,X,154534419,G,A,A,0.000795,GL000172.2,10058813,q28
6822,rs1050829,rs1050829,X,154535277,T,M,C,0.094570,GL000172.2,10059671,q28


In [10]:
pharmgkb_all_df = pd.DataFrame()
for i in range(len(pharmgkb_df)):
    chemicals = pharmgkb_df['chemicals'][i]
    rsid = pharmgkb_df['variant'][i]
    gene_symbol = pharmgkb_df['gene'][i].split(',')[0]
    label = pharmgkb_df['type'][i]
    disease_name = pharmgkb_df['phenotypes'][i]
    
    try:
        variant = variant_table[variant_table['rsid'] == rsid]['variant'].values[0]
    
    except:
        # print(rsid)
        variant = rsid

    try:
        chemical_type = chemicals_df[chemicals_df['Name'] == chemicals]['Type'].values[0]
        chemical_smile = chemicals_df[chemicals_df['Name'] == chemicals]['SMILES'].values[0]
        pubchem_id = chemicals_df[chemicals_df['Name'] == chemicals]['PubChem Compound Identifiers'].values[0]
        # print(chemicals_df[chemicals_df['Name'] == chemicals]['PubChem Compound Identifiers'].values[0])
    except :
        # print(chemicals)  
        chemical_type = None
        chemical_smile = None
        pubchem_id = None

    try:

        chromosome = gen_coord_df[gen_coord_df['dbSNP'] == rsid]['Chromosome'].values[0]
        position = gen_coord_df[gen_coord_df['dbSNP'] == rsid]['Position'].values[0]
        ref_allele = gen_coord_df[gen_coord_df['dbSNP'] == rsid]['REF Allele'].values[0]
        alt_allele = gen_coord_df[gen_coord_df['dbSNP'] == rsid]['ALT Allele (IUPAC)'].values[0]
        

    except IndexError:

        chromosome = None
        position = None
        ref_allele = None
        alt_allele = None

    pharmgkb_all_df = pharmgkb_all_df.append([{'data source': 'pharmgkb', 'chemical_type': chemical_type, 'chemical name': chemicals, 'pubchem id': pubchem_id, 'smile': chemical_smile, 'disease name': disease_name, 'gene symbol': gene_symbol, 'variant': variant, 'chromosome': chromosome, 'position': position, 'ref allele': ref_allele, 'alt allele': alt_allele, 'label': label}], ignore_index = True)

pharmgkb_all_df.to_csv('middlefile/pharmgkb_all_table.tsv', sep = '\t')
pharmgkb_all_df

,data source,chemical_type,chemical name,pubchem id,smile,disease name,gene symbol,variant,chromosome,position,ref allele,alt allele,label
0,pharmgkb,Drug,meloxicam,5281106,CC1=CN=C(S1)N/C(=C\2/C(=O)C3=CC=CC=C3S(=O)(=O)...,NaN,CYP2C9,"CYP2C9*1, CYP2C9*2, CYP2C9*3, CYP2C9*13",None,None,None,None,Metabolism/PK
1,pharmgkb,Drug,lornoxicam,5282204,CN1/C(=C(\NC2=CC=CC=N2)/O)/C(=O)C3=C(S1(=O)=O)...,NaN,CYP2C9,"CYP2C9*1, CYP2C9*3, CYP2C9*13",None,None,None,None,Metabolism/PK
2,pharmgkb,Drug,siponimod,44599207,NaN,NaN,CYP2C9,"CYP2C9*1, CYP2C9*2, CYP2C9*3",None,None,None,None,Metabolism/PK
3,pharmgkb,Prodrug,capecitabine,60953,CCCCCOC(=O)NC1=NC(=O)N(C=C1F)[C@H]2[C@@H]([C@@...,Neoplasms,DPYD,rs17376848,1,97450068,A,G,Toxicity
4,pharmgkb,Prodrug,capecitabine,60953,CCCCCOC(=O)NC1=NC(=O)N(C=C1F)[C@H]2[C@@H]([C@@...,Neoplasms,DPYD,M166V,1,97699535,T,C,Toxicity
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4823,pharmgkb,Drug,sertraline,68617,CN[C@H]1CC[C@H](C2=CC=CC=C12)C3=CC(=C(C=C3)Cl)Cl,"Depressive Disorder, Major,Panic Disorder",SLC6A4,"SLC6A4 HTTLPR long form (L allele), SLC6A4 HTT...",None,None,None,None,Efficacy
4824,pharmgkb,Drug,sertraline,68617,CN[C@H]1CC[C@H](C2=CC=CC=C12)C3=CC(=C(C=C3)Cl)Cl,"Anxiety Disorders,Depressive Disorder, Major",SLC6A4,"SLC6A4 HTTLPR long form (L allele), SLC6A4 HTT...",None,None,None,None,Toxicity
4825,pharmgkb,Drug,fluoxetine,3386,CNCCC(C1=CC=CC=C1)OC2=CC=C(C=C2)C(F)(F)F,"Depressive Disorder, Major",SLC6A4,rs25531,17,30237328,T,C,Efficacy
4826,pharmgkb,Drug,fluoxetine,3386,CNCCC(C1=CC=CC=C1)OC2=CC=C(C=C2)C(F)(F)F,"Depressive Disorder, Major,Obsessive-Compulsiv...",SLC6A4,"SLC6A4 HTTLPR long form (L allele), SLC6A4 HTT...",None,None,None,None,Efficacy


In [11]:
civic_df = pd.read_csv('rawData/CIViC/nightly-ClinicalEvidenceSummaries.tsv', sep='\t')
molecular_df = pd.read_csv('rawData/CIViC/nightly-MolecularProfileSummaries.tsv', sep='\t')
variant_df = pd.read_csv('rawData/CIViC/nightly-VariantSummaries.tsv', sep='\t')

variant_df['variant_id'] = variant_df['variant_id'].astype(object)

print(civic_df.shape)
civic_df = civic_df.dropna(subset=['therapies']).reset_index(drop=True)
print(civic_df.shape)
# civic_df['therapies'].isna().value_counts()

(4196, 25)
(2562, 25)


In [12]:
civic_all_df = pd.DataFrame()
for i in range(len(civic_df)):
    therapies = civic_df['therapies'][i]
    disease_name = civic_df['disease'][i]
    label = civic_df['significance'][i]
    molecular_profile = civic_df['molecular_profile'][i]
    # print(molecular_profile)
    try:
        variant_id = int(molecular_df[molecular_df['name'] == molecular_profile]['variant_ids'].values[0])
        # print(type(variant_id))
        # print(type(variant_df['variant_id'][24]))
        gene = variant_df[variant_df['variant_id'] == variant_id]['gene'].values[0]
        variant = variant_df[variant_df['variant_id'] == variant_id]['variant'].values[0]
        # print(gene)
    except:
        print(molecular_profile)
        continue

    civic_all_df = civic_all_df.append([{'data source': 'civic', 'chemical name': therapies, 'disease name': disease_name, 'gene symbol': gene, 'variant': variant, 'label': label}], ignore_index = True)

civic_all_df.to_csv('middlefile/civic_all_table.tsv', sep = '\t', )

BRAF V600E AND BRAF V600M
BRAF V600E AND BRAF Amplification
ALK EML4::ALK AND ALK L1196M
ABL1 BCR::ABL AND ABL1 E255K
ABL1 BCR::ABL AND ABL1 T315I
ABL1 BCR::ABL AND ABL1 T315I
ALK EML4::ALK AND ALK C1156Y
ALK EML4::ALK AND ALK L1196M
FLT3 ITD AND FLT3 D835Y
ALK EML4::ALK AND ALK Amplification
ALK EML4::ALK AND ALK G1202R
ALK EML4::ALK AND ALK L1196M
ALK EML4::ALK AND ALK S1206Y
ALK EML4::ALK AND ALK T1151dup
ABL1 BCR::ABL AND ABL1 F317L
ABL1 BCR::ABL AND ABL1 F317L
ABL1 BCR::ABL AND ABL1 T315I
ALK EML4::ALK AND ALK L1152R
ALK EML4::ALK AND ALK G1269A
ALK EML4::ALK AND ALK G1269A
ALK EML4::ALK AND ALK Amplification
ALK EML4::ALK AND ALK Amplification
EGFR Amplification AND EGFR EGFRVIII
ALK EML4::ALK AND ALK C1156Y
ALK EML4::ALK AND ALK C1156Y
ALK EML4::ALK AND ALK L1198F AND ALK C1156Y
ALK EML4::ALK AND ALK L1198F AND ALK C1156Y
MET Amplification AND MET Splice Site (c.3028G>A)
ALK EML4::ALK AND ALK V1180L
ALK EML4::ALK AND ALK V1180L
ALK EML4::ALK AND ALK V1180L
ALK EML4::ALK AND ALK 

In [13]:
dbmcs_df1 = pd.read_csv('rawData/dbMCS/Data for resistance.csv')
dbmcs_df2 = pd.read_csv('rawData/dbMCS/Data for sensitivity.csv')
print(dbmcs_df1.shape, dbmcs_df2.shape)

dbmcs_df = pd.concat([dbmcs_df1, dbmcs_df2]).reset_index(drop=True)
print(dbmcs_df.shape)
dbmcs_df.head()

(1164, 7) (2555, 7)
(3719, 7)


,Gene,Entrez_id,cDNA,Drug,Drug_id,Association,Evidence level
0,ABL1,25,c.944C>T,Nilotinib,DB04868,Resistant,1
1,ABL1,25,c.944C>T,Dasatinib,DB01254,Resistant,1
2,ABL1,25,c.944C>T,Bosutinib,DB06616,Resistant,1
3,ABL1,25,c.895G>C,Dasatinib,DB01254,Resistant,1
4,ABL1,25,c.943A>G,Dasatinib,DB01254,Resistant,1


In [14]:
dbmcs_all_df = pd.DataFrame()
for i in range(len(dbmcs_df)):
    therapies = dbmcs_df['Drug'][i]
    gene = dbmcs_df['Gene'][i]
    variant = dbmcs_df['cDNA'][i]
    label = dbmcs_df['Association'][i]

    dbmcs_all_df = dbmcs_all_df.append([{'data source': 'dbmcs', 'chemical name': therapies, 'gene symbol': gene, 'variant': variant, 'label': label}], ignore_index = True)
    
dbmcs_all_df.to_csv('middlefile/dbmcs_all_table.tsv', sep = '\t')

In [15]:
aimms_df = pd.read_csv('rawData/AIMMS/AIMMS-human.csv')
aimms_df
aimms_df['gene'].unique()

array(['ABL1', 'EGFR', 'AKR1B1'], dtype=object)

In [16]:
aimms_all_df = pd.DataFrame()
for i in range(len(aimms_df)):
    therapies = aimms_df['drug'][i]
    gene = aimms_df['gene'][i]
    variant = aimms_df['Mutants'][i]
    label = aimms_df['ddg'][i]

    aimms_all_df = aimms_all_df.append([{'data source': 'aimms', 'chemical name': therapies, 'gene symbol': gene, 'variant': variant, 'ddg': label}], ignore_index = True)
    
aimms_all_df.to_csv('middlefile/aimms_all_table.tsv', sep = '\t')

In [17]:
ret_df = pd.read_csv('rawData/RET/RET.csv')
ret_df.head()

,drug,Mutation,ic50,Unnamed: 3,Unnamed: 4,ddg,gene
0,Cabozantinib,L730I,2.96,12.333333,2.512306,1.496993,RET
1,Cabozantinib,L730V,2.37,9.875000,2.290006,1.364533,RET
2,Cabozantinib,L730V/V804M,7.37,30.708333,3.424534,2.040558,RET
3,Cabozantinib,E732K,1.15,4.791667,1.566878,0.933647,RET
4,Cabozantinib,V738A,1.20,5.000000,1.609438,0.959007,RET


In [18]:
ret_all_df = pd.DataFrame()
for i in range(len(ret_df)):
    therapies = ret_df['drug'][i]
    gene = ret_df['gene'][i]
    variant = ret_df['Mutation'][i]
    label = ret_df['ddg'][i]

    ret_all_df = ret_all_df.append([{'data source': 'ret', 'chemical name': therapies, 'gene symbol': gene, 'variant': variant, 'ddg': label}], ignore_index = True)
    
ret_all_df.to_csv('middlefile/ret_all_table.tsv', sep = '\t')

In [19]:
gdsc_df = pd.read_csv('rawData/GDSC/GDSC_drug_cell_line_sensitivity.tsv', sep = '\t')
gdsc_df.head()

,TARGET,DRUG,MUTATION,CELL_LINE_WT,CELL_LINE_MT,LN_IC50_WT,LN_IC50_MT,DDG,UNIPROT_ID
0,ABL1,Nilotinib,R608H,5637+A375+AMO-1+ASH-3+BE-13+BT-474+BxPC-3+CAL-...,A253,2.767916,3.384217,0.365149,P00519
1,ABL1,Nilotinib,A1045T,5637+A375+AMO-1+ASH-3+BE-13+BT-474+BxPC-3+CAL-...,C32,2.767916,3.814408,0.620030,P00519
2,ABL1,Nilotinib,V587M,5637+A375+AMO-1+ASH-3+BE-13+BT-474+BxPC-3+CAL-...,CA46,2.767916,3.014400,0.146038,P00519
3,ABL1,Nilotinib,P460L,5637+A375+AMO-1+ASH-3+BE-13+BT-474+BxPC-3+CAL-...,CAL-27,2.767916,3.602492,0.494474,P00519
4,ABL1,Nilotinib,G1079D,5637+A375+AMO-1+ASH-3+BE-13+BT-474+BxPC-3+CAL-...,CAPAN-2,2.767916,5.464567,1.597725,P00519


In [20]:
gdsc_all_df = pd.DataFrame()
for i in range(len(gdsc_df)):
    therapies = gdsc_df['DRUG'][i]
    gene = gdsc_df['TARGET'][i]
    variant = gdsc_df['MUTATION'][i]
    label = gdsc_df['DDG'][i]                                                                                    

    
    uniprot_id = gdsc_df['UNIPROT_ID'][i]

    gdsc_all_df = gdsc_all_df.append([{'data source': 'gdsc', 'chemical name': therapies, 'gene symbol': gene, 'variant': variant, 'uniprot id': uniprot_id, 'ddg': label}], ignore_index = True)
    
gdsc_all_df.to_csv('middlefile/gdsc_all_table.tsv', sep = '\t')

In [21]:
depmep_df = pd.read_csv('rawData/DepMap/MepMap_drug_cell_line_sensitivity.tsv', sep = '\t')
depmep_df.head()

,TARGET,DRUG,MUTATION,CELL_LINE_WT,CELL_LINE_MT,LN_IC50_WT,LN_IC50_MT,DDG
0,ALK,TAE-684,p.A96T,L428+HCT116+RVH421+DKMG+MG63+LC1SQSF+SUIT2+EN+...,C2BBE1,3.608218,1.563514,-1.211456
1,ALK,TAE-684,p.R317T,L428+HCT116+RVH421+DKMG+MG63+LC1SQSF+SUIT2+EN+...,CORL23,3.608218,7.618295,2.375909
2,ALK,TAE-684,p.G882C,L428+HCT116+RVH421+DKMG+MG63+LC1SQSF+SUIT2+EN+...,EFO27,3.608218,8.000000,2.602064
3,ALK,TAE-684,p.G689R,L428+HCT116+RVH421+DKMG+MG63+LC1SQSF+SUIT2+EN+...,G361,3.608218,4.944404,0.791670
4,ALK,TAE-684,p.R753Q,L428+HCT116+RVH421+DKMG+MG63+LC1SQSF+SUIT2+EN+...,HCC78,3.608218,8.000000,2.602064


In [22]:
depmep_all_df = pd.DataFrame()
for i in range(len(depmep_df)):
    therapies = depmep_df['DRUG'][i]
    gene = depmep_df['TARGET'][i]
    variant = depmep_df['MUTATION'][i]
    label = depmep_df['DDG'][i]                                                                                    


    depmep_all_df = depmep_all_df.append([{'data source': 'depmep', 'chemical name': therapies, 'gene symbol': gene, 'variant': variant, 'ddg': label}], ignore_index = True)
    
depmep_all_df.to_csv('middlefile/depmep_all_table.tsv', sep = '\t')

In [23]:
all_data_df = pd.concat([aimms_all_df, civic_all_df, dbmcs_all_df, depmep_all_df, gdsc_all_df, pharmgkb_all_df, ret_all_df]).reset_index(drop = True)
all_data_df.to_csv('middlefile/all_data_table.tsv', sep = '\t', index= False)


In [24]:
all_data_df = pd.read_csv('middlefile/all_data_table.tsv', sep = '\t')
symbol_list = all_data_df['gene symbol'].unique()
# for i in range(len(symbol_list)):
for i in tqdm(range(len(symbol_list))):
    try:
        gene = symbol_list[i]
        url = 'https://rest.uniprot.org/uniprotkb/search?format=json&query=(reviewed:true)%20AND%20(organism_id:9606)%20AND%20' + gene
        response = requests.get(url)
        data = response.json()
        all_data_df.loc[all_data_df['gene symbol'] == gene, 'uniprot id'] = data['results'][0]['primaryAccession']
        # print(gene)
        # print(data['results'][0]['primaryAccession'])

    except:
        # print(i)
        print(gene)
        continue
all_data_df.to_csv('middlefile/uniprot_table.tsv', sep = '\t', index= False)
all_data_df

  0%|          | 0/1323 [00:00<?, ?it/s]

 38%|███▊      | 498/1323 [26:33<44:40,  3.25s/it]  

OR52J2P


 50%|█████     | 664/1323 [34:42<28:22,  2.58s/it]  

PSORS1C3


...

MIR582


 94%|█████████▍| 1246/1323 [1:02:54<02:29,  1.94s/it]

CYP2A7P1


100%|██████████| 1323/1323 [1:06:12<00:00,  3.00s/it]


,data source,chemical name,gene symbol,variant,ddg,disease name,label,uniprot id,chemical_type,pubchem id,smile,chromosome,position,ref allele,alt allele
0,aimms,dasatinib,ABL1,M244V,0.290000,NaN,NaN,P00519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aimms,dasatinib,ABL1,G250E,0.480000,NaN,NaN,P00519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aimms,dasatinib,ABL1,Q252H,0.860000,NaN,NaN,P00519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aimms,dasatinib,ABL1,Y253F,0.330000,NaN,NaN,P00519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aimms,dasatinib,ABL1,Y253H,0.290000,NaN,NaN,P00519,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19664,ret,Nintedanib,RET,G810A,-0.044158,NaN,NaN,P07949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19665,ret,Nintedanib,RET,G810S,0.826043,NaN,NaN,P07949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19666,ret,Nintedanib,RET,V871I,0.149749,NaN,NaN,P07949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19667,ret,Nintedanib,RET,F998V,0.000000,NaN,NaN,P07949,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
gene = 'VEGFB'
url = 'https://rest.uniprot.org/uniprotkb/search?format=fasta&query=(reviewed:true)%20AND%20(organism_id:9606)%20AND%20' + gene
response = requests.get(url)
text = response.text
a = text.split('>sp')
a = a[1].split('\n')[1:]
a = ''.join(a)
a

'MSPLLRRLLLAALLQLAPAQAPVSQPDAPGHQRKVVSWIDVYTRATCQPREVVVPLTVELMGTVAKQLVPSCVTVQRCGGCCPDDGLECVPTGQHQVRMQILMIRYPSSQLGEMSLEEHSQCECRPKKKDSAVKPDRAATPHHRPQPRSVPGWDSAPGAPSPADITHPTPAPGPSAHAAPSTTSALTPGPAAAAADAAASSVAKGGA'

In [26]:
all_data_df = pd.read_csv('middlefile/uniprot_table.tsv', sep = '\t')
symbol_list = all_data_df['gene symbol'].unique()
symbol_list = [item for item in symbol_list if ',' not in item]
for i in tqdm(range(len(symbol_list))):
    try:
        gene = symbol_list[i]
        url = 'https://rest.uniprot.org/uniprotkb/search?format=fasta&query=(reviewed:true)%20AND%20(organism_id:9606)%20AND%20' + gene
        response = requests.get(url)
        text = response.text
        fasta = text.split('>sp')[1].split('\n')[1:]
        fasta_merged = ''.join(fasta)
        all_data_df.loc[all_data_df['gene symbol'] == gene, 'fasta'] = fasta_merged

    except:
        print(gene)
        continue

all_data_df.to_csv('middlefile/fasta_table.tsv', sep = '\t', index= False)
all_data_df

 35%|███▌      | 467/1323 [09:49<17:37,  1.24s/it]

SULT1C4


 38%|███▊      | 498/1323 [10:26<17:11,  1.25s/it]

OR52J2P


 50%|█████     | 664/1323 [14:04<14:29,  1.32s/it]

...

 86%|████████▋ | 1143/1323 [23:59<03:40,  1.23s/it]

MIR582


 94%|█████████▍| 1246/1323 [26:05<01:33,  1.22s/it]

CYP2A7P1


100%|██████████| 1323/1323 [27:40<00:00,  1.25s/it]


In [3]:
all_data_df = pd.read_csv('middlefile/fasta_table.tsv', sep = '\t')
drug_list = all_data_df['chemical name'].unique()
drug_list = [item for item in drug_list if ',' not in item]
for i in tqdm(range(len(drug_list))):
    drug = drug_list[i]
    try:
        # 通过药物名称查询相关信息
        compound = pcp.get_compounds(drug, 'name')[0]
        
        # 获取PubChem ID
        pubchem_id = compound.cid
        
        # 获取指纹 (Fingerprint)
        fingerprint = compound.cactvs_fingerprint
        
        # 获取SMILES
        smiles = compound.isomeric_smiles
    except:
        # print(drug)
        compound = None
        pubchem_id = None
        fingerprint = None
        smiles = None
        

    all_data_df.loc[all_data_df['chemical name'] == drug, 'pubchem id'] = str(pubchem_id)
    all_data_df.loc[all_data_df['chemical name'] == drug, 'pubchem fingerprint'] = fingerprint
    all_data_df.loc[all_data_df['chemical name'] == drug, 'smile'] = smiles


    

all_data_df.to_csv('middlefile/drug_table.tsv', sep = '\t', index= False)
all_data_df

  1%|          | 11/1417 [00:18<37:37,  1.61s/it]

Peginterferon Alfa-2b


  2%|▏         | 22/1417 [00:36<36:47,  1.58s/it]

JAK2 Inhibitor AZD1480


  2%|▏         | 24/1417 [00:39<35:25,  1.53s/it]

...

 96%|█████████▋| 1364/1417 [37:47<01:24,  1.60s/it]

peginterferon alfa-2a


 97%|█████████▋| 1374/1417 [38:03<01:08,  1.60s/it]

direct acting antivirals


100%|██████████| 1417/1417 [39:16<00:00,  1.66s/it]


In [9]:
all_data_df = pd.read_csv('middlefile/drug_table.tsv', sep = '\t')
disease_list = all_data_df['disease name'].unique()

token_endpoint = 'https://icdaccessmanagement.who.int/connect/token'
client_id = '8ce4ac1c-84d5-4e46-b686-796ab943d907_ce952a66-3b10-41a3-9da3-131c7739883a'
client_secret = 'ajUCkbpjrbzTSUmtp8cLtVHCIL6FjPFwz/dONYQeyAM='
scope = 'icdapi_access'
grant_type = 'client_credentials'

# get the OAUTH2 token

# set data to post
payload = {'client_id': client_id, 
	   	   'client_secret': client_secret, 
           'scope': scope, 
           'grant_type': grant_type}
           
# make request
r = requests.post(token_endpoint, data=payload, verify=False).json()
token = r['access_token']

# HTTP header fields to set
headers = {'Authorization':  'Bearer '+token, 
           'Accept': 'json', 
           'Accept-Language': 'en',
	   'API-Version': 'v2'}

for i in tqdm(range(len(disease_list))):
    disease = disease_list[i]
    try:
        # access ICD API
        uri = 'https://id.who.int/icd/entity/search?q=' + disease
        r = requests.get(uri, headers=headers, verify=False)
        icd11_id = r.json()['destinationEntities'][0]['id']

    except:
        # print(disease)
        icd11_id = None
        

    all_data_df.loc[all_data_df['disease name'] == disease, 'icd11'] = icd11_id

    
all_data_df.to_csv('middlefile/icd11_table.tsv', sep = '\t', index= False)
all_data_df['icd11'].value_counts() 


  0%|          | 0/844 [00:00<?, ?it/s]

nan


  2%|▏         | 16/844 [00:21<19:22,  1.40s/it]

T-cell Lymphoblastic Leukemia/lymphoma


  2%|▏         | 17/844 [00:22<19:28,  1.41s/it]

...

100%|█████████▉| 842/844 [19:56<00:02,  1.40s/it]

Drug Hypersensitivity,drug reaction with eosinophilia and systemic symptoms,Epidermal Necrolysis, Toxic,Maculopapular Exanthema,severe cutaneous adverse reactions,Stevens-Johnson Syndrome


100%|██████████| 844/844 [19:59<00:00,  1.42s/it]

Depressive Disorder, Major,Panic Disorder


http://id.who.int/icd/entity/1969743250    401
http://id.who.int/icd/entity/774170412     327
http://id.who.int/icd/entity/596808334     195
http://id.who.int/icd/entity/1669279433    176
http://id.who.int/icd/entity/488336723     168
                                          ... 
http://id.who.int/icd/entity/146744831       1
http://id.who.int/icd/entity/190029283       1
http://id.who.int/icd/entity/154709545       1
http://id.who.int/icd/entity/944754984       1
http://id.who.int/icd/entity/1890374210      1
Name: icd11, Length: 357, dtype: int64

In [39]:
all_data_df = pd.read_csv('middlefile/icd11_table.tsv', sep = '\t')
uniprot_id_list = all_data_df['uniprot id'].unique()
for i in tqdm(range(len(uniprot_id_list))):
    uniprot_id = uniprot_id_list[i]
    search_url = "https://search.rcsb.org/rcsbsearch/v2/query"
    search_request = {
        "query": {
            "type": "group",
            "logical_operator": "and",
            "nodes": [
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "operator": "exact_match",
                        "value": uniprot_id,
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession"
                    }
                },
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "operator": "exact_match",
                        "value": "UniProt",
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_name"
                    }
                }
            ]
        },
        "request_options": {
            "paginate": {
            "start": 0,
            "rows": 1000
            }
        },
        "return_type": "entry"
    }
    try:
        response = requests.post(search_url, json=search_request)
        pdb_list = response.json()['result_set']
        pdb_str = ''
        for i, pdb_item in enumerate(pdb_list):
            pdb_str += pdb_item['identifier']
            if i != len(pdb_list)-1 :
                pdb_str += ','
        # print(uniprot_id)
        # print(pdb_str)
    except:
        # print(uniprot_id)
        pdb_list = None

    all_data_df.loc[all_data_df['uniprot id'] == uniprot_id, 'pdb list'] = pdb_str   

all_data_df.to_csv('middlefile/pdb_table.tsv', sep = '\t', index= False)
all_data_df                              


100%|██████████| 1296/1296 [20:26<00:00,  1.06it/s]


,data source,chemical name,gene symbol,variant,ddg,disease name,label,uniprot id,chemical_type,pubchem id,smile,chromosome,position,ref allele,alt allele,fasta,pubchem fingerprint,icd11,pdb list
0,aimms,dasatinib,ABL1,M244V,0.290000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
1,aimms,dasatinib,ABL1,G250E,0.480000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
2,aimms,dasatinib,ABL1,Q252H,0.860000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
3,aimms,dasatinib,ABL1,Y253F,0.330000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
4,aimms,dasatinib,ABL1,Y253H,0.290000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19664,ret,Nintedanib,RET,G810A,-0.044158,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,"2IVS,2IVT,2IVU,2IVV,2X2K,2X2L,2X2M,2X2U,4CKI,4..."
19665,ret,Nintedanib,RET,G810S,0.826043,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,"2IVS,2IVT,2IVU,2IVV,2X2K,2X2L,2X2M,2X2U,4CKI,4..."
19666,ret,Nintedanib,RET,V871I,0.149749,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,"2IVS,2IVT,2IVU,2IVV,2X2K,2X2L,2X2M,2X2U,4CKI,4..."
19667,ret,Nintedanib,RET,F998V,0.000000,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,"2IVS,2IVT,2IVU,2IVV,2X2K,2X2L,2X2M,2X2U,4CKI,4..."


In [5]:
all_data_df = pd.read_csv('middlefile/pdb_table.tsv', sep = '\t')
all_data_df['id'] = range(1,len(all_data_df)+1)
all_data_df.to_csv('middlefile/final_table.tsv', sep = '\t', index= False)

In [4]:
all_data_df = pd.read_csv('middlefile/icd11_table.tsv', sep = '\t')
uniprot_id_list = all_data_df['uniprot id'].unique()
for i in tqdm(range(5)):
    uniprot_id = uniprot_id_list[i]
    search_url = "https://search.rcsb.org/rcsbsearch/v2/query"
    search_request = {
        "query": {
            "type": "group",
            "logical_operator": "and",
            "nodes": [
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "operator": "exact_match",
                        "value": uniprot_id,
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession"
                    }
                },
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "operator": "exact_match",
                        "value": "UniProt",
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_name"
                    }
                }
            ]
        },
        "request_options": {
            "paginate": {
            "start": 0,
            "rows": 1000
            }
        },
        "return_type": "entry"
    }
    try:
        response = requests.post(search_url, json=search_request)
        
        pdb_list = response.json()['result_set']
        print(response.json())
        pdb_str = ''
        for i, pdb_item in enumerate(pdb_list):
            pdb_str += pdb_item['identifier']
            if i != len(pdb_list)-1 :
                pdb_str += ','
        # print(uniprot_id)
        # print(pdb_str)
    except:
        # print(uniprot_id)
        pdb_list = None

    all_data_df.loc[all_data_df['uniprot id'] == uniprot_id, 'pdb list'] = pdb_str   

all_data_df.to_csv('middlefile/pdb_table.tsv', sep = '\t', index= False)
all_data_df                              


 20%|██        | 1/5 [00:01<00:04,  1.06s/it]

{'query_id': 'f4b8358d-0445-4b08-932b-d9090531624e', 'result_type': 'entry', 'total_count': 81, 'result_set': [{'identifier': '1AB2', 'score': 1.0}, {'identifier': '1AWO', 'score': 1.0}, {'identifier': '1BBZ', 'score': 1.0}, {'identifier': '1JU5', 'score': 1.0}, {'identifier': '1OPL', 'score': 1.0}, {'identifier': '1ZZP', 'score': 1.0}, {'identifier': '2ABL', 'score': 1.0}, {'identifier': '2E2B', 'score': 1.0}, {'identifier': '2F4J', 'score': 1.0}, {'identifier': '2FO0', 'score': 1.0}, {'identifier': '2G1T', 'score': 1.0}, {'identifier': '2G2F', 'score': 1.0}, {'identifier': '2G2H', 'score': 1.0}, {'identifier': '2G2I', 'score': 1.0}, {'identifier': '2GQG', 'score': 1.0}, {'identifier': '2HIW', 'score': 1.0}, {'identifier': '2HYY', 'score': 1.0}, {'identifier': '2HZ0', 'score': 1.0}, {'identifier': '2HZ4', 'score': 1.0}, {'identifier': '2HZI', 'score': 1.0}, {'identifier': '2O88', 'score': 1.0}, {'identifier': '2V7A', 'score': 1.0}, {'identifier': '3CS9', 'score': 1.0}, {'identifier': 

 40%|████      | 2/5 [00:02<00:03,  1.06s/it]

{'query_id': '26142bb3-6597-4296-8190-2447bd133ff5', 'result_type': 'entry', 'total_count': 291, 'result_set': [{'identifier': '1IVO', 'score': 1.0}, {'identifier': '1M14', 'score': 1.0}, {'identifier': '1M17', 'score': 1.0}, {'identifier': '1MOX', 'score': 1.0}, {'identifier': '1NQL', 'score': 1.0}, {'identifier': '1XKK', 'score': 1.0}, {'identifier': '1YY9', 'score': 1.0}, {'identifier': '1Z9I', 'score': 1.0}, {'identifier': '2EB2', 'score': 1.0}, {'identifier': '2EB3', 'score': 1.0}, {'identifier': '2GS2', 'score': 1.0}, {'identifier': '2GS6', 'score': 1.0}, {'identifier': '2GS7', 'score': 1.0}, {'identifier': '2ITN', 'score': 1.0}, {'identifier': '2ITO', 'score': 1.0}, {'identifier': '2ITP', 'score': 1.0}, {'identifier': '2ITQ', 'score': 1.0}, {'identifier': '2ITT', 'score': 1.0}, {'identifier': '2ITU', 'score': 1.0}, {'identifier': '2ITV', 'score': 1.0}, {'identifier': '2ITW', 'score': 1.0}, {'identifier': '2ITX', 'score': 1.0}, {'identifier': '2ITY', 'score': 1.0}, {'identifier':

 60%|██████    | 3/5 [00:03<00:02,  1.05s/it]

...

 80%|████████  | 4/5 [00:04<00:01,  1.05s/it]

{'query_id': '6a94955d-3799-4f72-b713-8e3e06915360', 'result_type': 'entry', 'total_count': 19, 'result_set': [{'identifier': '2QRV', 'score': 1.0}, {'identifier': '3A1A', 'score': 1.0}, {'identifier': '3A1B', 'score': 1.0}, {'identifier': '3LLR', 'score': 1.0}, {'identifier': '3SVM', 'score': 1.0}, {'identifier': '4QBQ', 'score': 1.0}, {'identifier': '4QBR', 'score': 1.0}, {'identifier': '4QBS', 'score': 1.0}, {'identifier': '4U7P', 'score': 1.0}, {'identifier': '4U7T', 'score': 1.0}, {'identifier': '5YX2', 'score': 1.0}, {'identifier': '6BRR', 'score': 1.0}, {'identifier': '6F57', 'score': 1.0}, {'identifier': '6PA7', 'score': 1.0}, {'identifier': '6W89', 'score': 1.0}, {'identifier': '6W8B', 'score': 1.0}, {'identifier': '6W8D', 'score': 1.0}, {'identifier': '6W8J', 'score': 1.0}, {'identifier': '8BA5', 'score': 1.0}]}


100%|██████████| 5/5 [00:05<00:00,  1.05s/it]

{'query_id': 'dc981724-9284-4979-ad33-6dd5aca8deff', 'result_type': 'entry', 'total_count': 69, 'result_set': [{'identifier': '1S9J', 'score': 1.0}, {'identifier': '2P55', 'score': 1.0}, {'identifier': '3DV3', 'score': 1.0}, {'identifier': '3DY7', 'score': 1.0}, {'identifier': '3E8N', 'score': 1.0}, {'identifier': '3EQB', 'score': 1.0}, {'identifier': '3EQC', 'score': 1.0}, {'identifier': '3EQD', 'score': 1.0}, {'identifier': '3EQF', 'score': 1.0}, {'identifier': '3EQG', 'score': 1.0}, {'identifier': '3EQH', 'score': 1.0}, {'identifier': '3EQI', 'score': 1.0}, {'identifier': '3MBL', 'score': 1.0}, {'identifier': '3ORN', 'score': 1.0}, {'identifier': '3OS3', 'score': 1.0}, {'identifier': '3PP1', 'score': 1.0}, {'identifier': '3SLS', 'score': 1.0}, {'identifier': '3V01', 'score': 1.0}, {'identifier': '3V04', 'score': 1.0}, {'identifier': '3VVH', 'score': 1.0}, {'identifier': '3W8Q', 'score': 1.0}, {'identifier': '3WIG', 'score': 1.0}, {'identifier': '3ZLS', 'score': 1.0}, {'identifier': 

,data source,chemical name,gene symbol,variant,ddg,disease name,label,uniprot id,chemical_type,pubchem id,smile,chromosome,position,ref allele,alt allele,fasta,pubchem fingerprint,icd11,pdb list
0,aimms,dasatinib,ABL1,M244V,0.290000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
1,aimms,dasatinib,ABL1,G250E,0.480000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
2,aimms,dasatinib,ABL1,Q252H,0.860000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
3,aimms,dasatinib,ABL1,Y253F,0.330000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
4,aimms,dasatinib,ABL1,Y253H,0.290000,NaN,NaN,P00519,NaN,3062316,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,NaN,NaN,NaN,NaN,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1110000001111011101100000000000001000100000000...,NaN,"1AB2,1AWO,1BBZ,1JU5,1OPL,1ZZP,2ABL,2E2B,2F4J,2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19664,ret,Nintedanib,RET,G810A,-0.044158,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,NaN
19665,ret,Nintedanib,RET,G810S,0.826043,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,NaN
19666,ret,Nintedanib,RET,V871I,0.149749,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,NaN
19667,ret,Nintedanib,RET,F998V,0.000000,NaN,NaN,P07949,NaN,135423438,CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...,NaN,NaN,NaN,NaN,MAKATSGAAGLRLLLLLLLPLLGKVALGLYFSRDAYWEKLYVDQAA...,1111000001111011101110000000000000000000000000...,NaN,NaN


In [12]:
search_url = "https://search.rcsb.org/rcsbsearch/v2/query"
search_request = {
  "query": {
    "type": "terminal",
    "service": "sequence",
    "parameters": {
      "evalue_cutoff": 1,
      "identity_cutoff": 0.9,
      "sequence_type": "protein",
      "value": "MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSSTAFIPLISTRVSLRKTRQPPERIASGAITKGVVLDSTEALCLAISRNSEQMASHSAVLEAGKNLYTFCVSYVDSIQQMRNKFAFREAINKLENNLRELQICPATAGSGPAATQDFSKLLSSVKEISDIVQR"
    }
  },
  "request_options": {
    "scoring_strategy": "sequence"
  },
  "return_type": "polymer_entity"
}
response = requests.post(search_url, json=search_request)
print(response.json())

{'query_id': 'e4bfe2df-0dba-44b1-8d9c-4aac2366e493', 'result_type': 'polymer_entity', 'total_count': 104, 'result_set': [{'identifier': '5MO4_1', 'score': 1.0}, {'identifier': '1OPL_1', 'score': 0.9966960352422908}, {'identifier': '1OPK_1', 'score': 0.9955947136563876}, {'identifier': '2FO0_1', 'score': 0.974669603524229}, {'identifier': '8SSN_1', 'score': 0.9085903083700441}, {'identifier': '4XEY_1', 'score': 0.789647577092511}, {'identifier': '2E2B_1', 'score': 0.5374449339207048}, {'identifier': '6XR6_1', 'score': 0.5374449339207048}, {'identifier': '6XR7_1', 'score': 0.5374449339207048}, {'identifier': '4WA9_1', 'score': 0.5330396475770925}]}


In [6]:
import pandas as pd
data_df = pd.read_csv('middlefile/final_table.tsv', sep = '\t')
data_df['data source'].value_counts().reset_index()

,data source,count
0,gdsc,8442
1,pharmgkb,4828
2,dbmcs,3719
3,civic,2381
4,depmep,193
5,ret,56
6,aimms,50


In [7]:
print(data_df.groupby('data source')['disease name'].nunique().reset_index())
print(data_df['disease name'].nunique())

  data source  disease name
0       aimms             0
1       civic           181
2       dbmcs             0
3      depmep             0
4        gdsc             0
5    pharmgkb           666
6         ret             0
843


In [10]:
print(data_df.groupby('data source')['gene symbol'].nunique())
print(data_df['gene symbol'].nunique())

data source
aimms          3
civic        256
dbmcs        112
depmep         5
gdsc         124
pharmgkb    1054
ret            1
Name: gene symbol, dtype: int64
1323


In [11]:
print(data_df.groupby('data source')['chemical name'].nunique())
print(data_df['chemical name'].nunique())

data source
aimms         4
civic       623
dbmcs       628
depmep        5
gdsc        234
pharmgkb    842
ret           4
Name: chemical name, dtype: int64
2142


In [12]:
def count_mutation_types(group):
    # 定义单点突变的正则表达式
    single_mutation_pattern = '^[A-Z][0-9]*[A-Z]$|^c\.[0-9]+[A-Z]>[A-Z]$|^p\.[A-Z][0-9]+[A-Z]$'
    
    # 统计单点和多点突变的数量
    single_point_mutations = group['variant'].str.match(single_mutation_pattern).sum()
    multi_point_mutations = len(group) - single_point_mutations
    
    # 创建一个包含统计结果的 Series
    result = pd.Series({
        '单点突变数量': single_point_mutations,
        '多点突变数量': multi_point_mutations
    })
    
    return result

# 按照 data source 分组，并应用 count_mutation_types 函数
mutation_counts = data_df.groupby('data source').apply(count_mutation_types)

# 打印结果
print(mutation_counts)
print(count_mutation_types(data_df))

             单点突变数量  多点突变数量
data source                
aimms            47       3
civic           820    1561
dbmcs          2986     733
depmep          173      20
gdsc           7114    1328
pharmgkb       1524    3304
ret              52       4
单点突变数量    12716
多点突变数量     6953
dtype: int64
